# 2. Modelado de Lenguaje

Veremos cómo armar un modelo de n-gramas y usarlo para generar lenguaje natural.

## 2.1. Contando N-Gramas

In [ ]:
from nltk.corpus import gutenberg
sents = list(gutenberg.sents('austen-emma.txt'))

Primero vemos cómo imprimir todos los trigramas de una sola oración:

In [ ]:
sent = sents[0]

n = 3  # trigramas
for i in range(len(sent) - n + 1):
    print(sent[i:i+n])

Ahora veamos cómo contar los trigramas de todas las oraciones:

In [ ]:
from collections import defaultdict

count = defaultdict(int)

for sent in sents:
    for i in range(len(sent) - n + 1):
        ngram = tuple(sent[i:i+n])  # los diccionarios no pueden guardar listas, pero sí tuplas
        count[ngram] += 1

In [ ]:
count

Este código sirve para n-gramas en general.

### Ejercicios

1. Agregar marcadores de principio y final de oración
2. Contar n-gramas y (n-1)-gramas al mismo tiempo.

## 2.2. Probabilidad de una Oración

Los siguientes conteos de unigramas y bigramas se calculan a partir de las siguientes dos oraciones:
- "el gato come pescado"
- "la gata come salmón"


In [ ]:
counts = {
    ('<s>',): 2,
    ('<s>', 'el'): 1,
    ('el',): 1,
    ('el', 'gato'): 1,
    ('gato',): 1,
    ('gato', 'come'): 1,
    ('come',): 2,
    ('come', 'pescado'): 1,
    ('pescado',): 1,
    ('pescado', '</s>'): 1,
    ('<s>', 'la'): 1,
    ('la',): 1,
    ('la', 'gata'): 1,
    ('gata',): 1,
    ('gata', 'come'): 1,
    ('come', 'salmón'): 1,
    ('salmón',): 1,
    ('salmón', '</s>'): 1,
}

La probabilidad de una palabra w2 dada la palabra anterior w1 es:

In [ ]:
def cond_prob(w2, w1):
    return counts[w1, w2] / counts[w1,]

cond_prob('salmón', 'come')

La probabilidad de una oración entera es:

In [ ]:
sent = 'la gata come pescado </s>'.split()

prev = '<s>'
prob = 1.0
for w in sent:
    prob *= cond_prob(w, prev)
    prev = w
    
prob

### Ejericicios:

3. Adaptar el código a n-gramas en general.
4. Agregar suavizado add-one.
5. Calcular probabilidades en espacio logarítmico (base 2).

## 2.3. Generando Lenguaje Natural

El siguiente modelo de bigramas se calcula a partir de las siguientes dos oraciones:
- "el gato come pescado"
- "la gata come salmón"


In [ ]:
probs = {
    '<s>': {'el': 0.5, 'la': 0.5},
    # '<s>': {'el': 0.6, 'la': 0.2, 'los': 0.1, 'las': 0.1},
    'el': {'gato': 1.0},
    'gato': {'come': 1.0},
    'come': {'pescado': 0.5, 'salmón': 0.5},
    'pescado': {'.': 1.0},
    '.': {'</s>': 1.0},
    'la': {'gata': 1.0},
    'gata': {'come': 1.0},
    'salmón': {'.': 1.0},
}

list(probs['<s>'].items())  # convertir un diccionario a lista de pares

Cada entrada del diccionario contiene una distribución discreta finita para la palabra siguiente dada la palabra anterior. Samplear de una distribución discreta finita es tan fácil como samplear un número al azar entre 0 y 1 y ver en qué región cae (ver [Wikipedia](https://en.wikipedia.org/wiki/Pseudo-random_number_sampling#Finite_discrete_distributions)).

Empezamos sampleando la primer palabra:

In [ ]:
from random import random

def sample(problist):
    r = random()  # entre 0 y 1
    i = 0
    word, prob = problist[0]
    acum = prob
    while r > acum:
        i += 1
        word, prob = problist[i]
        acum += prob
    
    return word

sample(list(probs['<s>'].items()))

Si lo repetimos muchas veces, podemos ver que el resultado del sampleo se corresponde con las probabilidades:

In [ ]:
results = [sample(list(probs['<s>'].items())) for i in range(1000)]

from collections import Counter
print(Counter(results))

**Observaciones:**
- Si se ordena la lista de probabilidades de mayor a menor, el sampling es más rápido.
- El sampling también se puede hacer usando [random.choices](https://docs.python.org/3/library/random.html#random.choices) de python
ó [random.choice](https://stackoverflow.com/questions/11373192/generating-discrete-random-variables-with-specified-weights-using-scipy-or-numpy) de numpy.

Ahora veamos cómo samplear una oración completa:

In [ ]:
word = '<s>'
while word != '</s>':
    problist = list(probs[word].items())
    word = sample(problist)
    print(word)

Acá se ve que se pueden generar oraciones nuevas (no vistas en tiempo de entrenamiento).

### Ejercicios

6. Calcular el modelo de n-gramas a partir de un corpus abritrario.
7. Adaptar el código a n-gramas en general: usar tuplas como claves en probs!
8. Precalcular las listas ordenadas de mayor a menor (ver sorted_prob en los tests)